In [21]:
import marko
from marko.block import Heading



In [22]:
from datetime import datetime
from typing import Union, List
from pathlib import Path

import marko
from marko.block import Heading
from marko.inline import RawText
import pandas as pd

def extract_headings_only(input_path: str, output_path: str) -> None:
    """
    Extracts only headings from a Markdown file and writes them to another file.
    """
    # Read the Markdown content
    input_text = Path(input_path).read_text(encoding='utf-8')

    # Parse the Markdown into an AST
    markdown_ast = marko.parse(input_text)

    # Collect formatted headings
    headings = []
    for node in markdown_ast.children:
        if isinstance(node, Heading):
            # Extract plain text content of heading
            heading_text = ''.join(child.children if hasattr(child, 'children') else child
                                   for child in node.children)
            headings.append(f"{'#' * node.level} {heading_text.strip()}")

    # Write output
    Path(output_path).write_text('\n\n'.join(headings), encoding='utf-8')
    print(f"Headings written to: {output_path}")

def extract_text(node) -> str:
    """
    Recursively extracts plain text from a marko node (including inline formatting).
    """
    if isinstance(node, RawText):
        return node.children
    if hasattr(node, 'children'):
        return ''.join(extract_text(child) for child in node.children)
    return ''

def parse_markdown(filepath: str, skip_first_heading: bool = False, column_map: dict = None) -> pd.DataFrame:
    """
    Parses a Markdown travel log file using marko to extract travel durations grouped by country and city.
    Splits visits to the same location into multiple entries if they appear non-consecutively.

    Parameters:
        filepath (str): Path to the .md file.
        skip_first_heading (bool): Whether to skip the first heading (e.g., title or introduction).
        column_map (dict): Optional mapping of output column names, e.g., {'country': 'Country', 'city': 'City', ...}

    Returns:
        pd.DataFrame: A DataFrame with columns [Country, City, Start, End, Duration] by default.
    """
    with open(filepath, encoding="utf-8") as f:
        content = f.read()

    ast = marko.parse(content)

    country = city = None
    date_entries = []
    heading_count = 0

    for node in ast.children:
        if not isinstance(node, Heading):
            continue

        heading_count += 1
        if skip_first_heading and heading_count == 1:
            continue

        level = node.level
        text = extract_text(node).strip()

        if level == 1:
            country = text
            city = None  # reset city
        elif level == 2:
            city = text
        elif level == 3:
            parts = text.split(", ")
            if len(parts) == 2:
                try:
                    date = datetime.strptime(parts[1], "%d.%m.%Y").date()
                    effective_city = city if city else country
                    date_entries.append({
                        "country": country,
                        "city": effective_city,
                        "date": date
                    })
                except ValueError:
                    continue  # malformed date

    # Sentry date entry

    # Sort by date to ensure order
    date_entries.sort(key=lambda x: x["date"])

    # Track city/country transitions
    visits = []
    if not date_entries:
        return pd.DataFrame(columns=column_map.values() if column_map else [])

    prev = date_entries[0]
    start_date = end_date = prev["date"]

    for i in range(1, len(date_entries)):
        entry = date_entries[i]
        is_last = i == len(date_entries) - 1

        if entry["country"] == prev["country"] and entry["city"] == prev["city"] and not is_last:
            end_date = entry["date"]  # extend visit
        else:
            visits.append({
                "country": prev["country"],
                "city": prev["city"],
                "start": start_date,
                "end": end_date,
                "duration": (end_date - start_date).days + 1
            })
            prev = entry
            start_date = end_date = entry["date"]

    # Default column mapping
    default_columns = {
        "country": "Country",
        "city": "City",
        "start": "Start",
        "end": "End",
        "duration": "Duration"
    }

    if column_map:
        columns = column_map
    else:
        columns = default_columns

    # Build DataFrame using column map
    df = pd.DataFrame([
        {
            columns["country"]: visit["country"],
            columns["city"]: visit["city"],
            columns["start"]: visit["start"],
            columns["end"]: visit["end"],
            columns["duration"]: visit["duration"]
        }
        for visit in visits
    ])

    return df

def summarize_travel(df: pd.DataFrame, column_map: dict = None, show_total: bool = True) -> pd.DataFrame:
    """
    Summarizes travel statistics per country:
    - Total duration (sum of days)
    - Number of unique cities visited
    - First date of entry
    - Last date of exit
    - (Optional) Total row with overall stats

    Parameters:
        df (pd.DataFrame): The travel DataFrame returned by parse_markdown.
        column_map (dict): Optional mapping of DataFrame column names.
        show_total (bool): Whether to include a final row with totals.

    Returns:
        pd.DataFrame: Summary table with optional total row.
    """
    default_columns = {
        "country": "Country",
        "city": "City",
        "duration": "Duration",
        "start": "Start",
        "end": "End"
    }

    columns = column_map if column_map else default_columns

    summary = df.groupby(columns["country"]).agg(
        Total_Days=(columns["duration"], "sum"),
        Cities_Visited=(columns["city"], pd.Series.nunique),
        Entry_Date=(columns["start"], "min"),
        Exit_Date=(columns["end"], "max")
    ).reset_index()

    # Sort by Entry_Date
    summary = summary.sort_values(by="Entry_Date").reset_index(drop=True)

    if show_total:
        total_row = {
            columns["country"]: "TOTAL",
            "Total_Days": summary["Total_Days"].sum(),
            "Cities_Visited": summary["Cities_Visited"].sum(),
            "Entry_Date": summary["Entry_Date"].min(),
            "Exit_Date": summary["Exit_Date"].max()
        }
        summary = pd.concat([summary, pd.DataFrame([total_row])], ignore_index=True)

    return summary

from geopy.geocoders import Nominatim
from geopy.exc import GeocoderUnavailable, GeocoderTimedOut
import time

def map_cities_to_coords(df: pd.DataFrame, city_col="City", country_col="Country", user_agent="city-geocoder", api_wait_time=0.1) -> pd.DataFrame:
    """
    Adds 'Latitude' and 'Longitude' columns to the DataFrame by geocoding city and country.
    If a city cannot be geocoded, sets both to None.

    Parameters:
        df (pd.DataFrame): Input DataFrame with city and country columns.
        city_col (str): Name of the column containing city names.
        country_col (str): Name of the column containing country names.
        user_agent (str): Identifier for the Nominatim geocoder.

    Returns:
        pd.DataFrame: DataFrame with additional 'Latitude' and 'Longitude' columns.
        :param api_wait_time:
    """
    geolocator = Nominatim(user_agent=user_agent)
    latitudes, longitudes = [], []

    for _, row in df.iterrows():
        location_str = f"{row[city_col]}, {row[country_col]}"
        try:
            location = geolocator.geocode(location_str, timeout=10)
            if location:
                latitudes.append(location.latitude)
                longitudes.append(location.longitude)
            else:
                latitudes.append(None)
                longitudes.append(None)
        except (GeocoderTimedOut, GeocoderUnavailable):
            latitudes.append(None)
            longitudes.append(None)
        time.sleep(api_wait_time)  # Be polite to the API

    df["Latitude"] = latitudes
    df["Longitude"] = longitudes
    return df

def get_unmapped_locations(df: pd.DataFrame, lat_col="Latitude", lon_col="Longitude") -> pd.DataFrame:
    """
    Returns a DataFrame of all entries where latitude or longitude is None.

    Parameters:
        df (pd.DataFrame): The DataFrame with geolocation columns.
        lat_col (str): Name of the latitude column.
        lon_col (str): Name of the longitude column.

    Returns:
        pd.DataFrame: Rows where coordinates are missing.
    """
    return df[df[lat_col].isna() | df[lon_col].isna()]

from typing import Union, List

def set_manual_coordinates_by_index(
    df: pd.DataFrame,
    updates: Union[List[Union[int, float]], List[List[Union[int, float]]]],
    lat_col: str = "Latitude",
    lon_col: str = "Longitude"
) -> pd.DataFrame:
    """
    Manually sets latitude and longitude for rows identified by their index.

    Parameters:
        df (pd.DataFrame): DataFrame to update.
        updates (list): Either a single update [index, lat, lon] or a list of such updates.
        lat_col (str): Name of the latitude column.
        lon_col (str): Name of the longitude column.

    Returns:
        pd.DataFrame: Updated DataFrame with manual coordinates set.
    """
    if isinstance(updates[0], (int, float)):
        updates = [updates]  # single update case

    for idx, lat, lon in updates:
        if idx in df.index:
            df.at[idx, lat_col] = lat
            df.at[idx, lon_col] = lon

    return df


import pandas as pd
from typing import Optional

def filter_visits(
    df: pd.DataFrame,
    country: Optional[str] = None,
    start_date: Optional[str] = None,
    end_date: Optional[str] = None,
    column_map: dict = None
) -> pd.DataFrame:
    """
    Filters a DataFrame of visit records by country, start date, and/or end date.

    Parameters:
        df (pd.DataFrame): The original DataFrame.
        column_map (dict): mapping for logical column-names; country, city, start, end, duration
        country (str, optional): If provided, filters rows where 'country' matches.
        start_date (str, optional): If provided, filters rows where 'start' is on or after this date (YYYY-MM-DD).
        end_date (str, optional): If provided, filters rows where 'end' is on or before this date (YYYY-MM-DD).

    Returns:
        pd.DataFrame: A filtered copy of the DataFrame.
    """
    # Copy the DataFrame to avoid modifying the original
    df_filtered = df.copy()

    # Default column mapping
    default_columns = {
        "country": "Country",
        "city": "City",
        "start": "Start",
        "end": "End",
        "duration": "Duration"
    }

    if column_map:
        columns = column_map
    else:
        columns = default_columns


    # Convert date columns to datetime for comparison
    df_filtered[columns["start"]] = pd.to_datetime(df_filtered[columns["start"]])
    df_filtered[columns["end"]] = pd.to_datetime(df_filtered[columns["end"]])

    # Apply filters conditionally
    if country:
        df_filtered = df_filtered[df_filtered[columns["country"]] == country]

    if start_date:
        start_date = pd.to_datetime(start_date, dayfirst=True)
        df_filtered = df_filtered[df_filtered[columns["start"]] >= start_date]

    if end_date:
        end_date = pd.to_datetime(end_date, dayfirst=True)
        df_filtered = df_filtered[df_filtered[columns["end"]] <= end_date]

    return df_filtered

In [23]:
import plotly.graph_objects as go
from plotly.colors import sample_colorscale
import pandas as pd
import numpy as np
from typing import Optional, Dict

import plotly.colors as pc

def get_rgb_from_scale(value: float, colorscale: str) -> tuple:
    """
    Convert a normalized value (0-1) into an RGB tuple using the given Plotly colorscale.

    Parameters:
        value (float): Normalized value between 0 and 1.
        colorscale (str): Name of the Plotly color scale.

    Returns:
        tuple: (r, g, b)
    """
    # Ensure value is in [0, 1]
    value = max(0, min(1, value))
    rgb_string = pc.find_intermediate_color(
        pc.get_colorscale(colorscale)[0][1],
        pc.get_colorscale(colorscale)[-1][1],
        value,
        colortype="rgb"
    )
    return tuple(int(x) for x in rgb_string.strip("rgb()").split(","))


def plot_travel_map(
    df: pd.DataFrame,
    lat_col: str = "Latitude",
    lon_col: str = "Longitude",
    city_col: str = "City",
    duration_col: str = "Duration",
    date_col: str = "Start",
    default_marker_size: int = 5,
    marker_scale: float = 2.0,
    color_scale: str = "Viridis",
    line_opacity: float = 0.6,
    auto_center: bool = False,
    plot_kwargs: Optional[Dict] = None
) -> go.Figure:
    """
    Plots a world map of visited cities with size based on duration and color based on entry date.

    Parameters:
        df (pd.DataFrame): DataFrame with geolocation and visit data.
        lat_col (str): Name of the latitude column.
        lon_col (str): Name of the longitude column.
        city_col (str): Name of the city column.
        duration_col (str): Name of the duration column.
        date_col (str): Name of the entry date column (should be datetime).
        default_marker_size (int): Minimum size for markers.
        marker_scale (float): Scaling factor for marker size.
        color_scale (str): Color scale name for Plotly.
        line_opacity (float): Opacity of the lines connecting cities.
        auto_center (bool): If True, zoom to the bounding box of the cities.
        plot_kwargs (dict): Additional keyword arguments to pass to `go.Figure`.

    Returns:
        go.Figure: The resulting Plotly figure.
    """
    if plot_kwargs is None:
        plot_kwargs = {}

    df_sorted = df.sort_values(by=date_col).reset_index(drop=True)

    # Normalize date to float for coloring
    date_min = df_sorted[date_col].min()
    date_max = df_sorted[date_col].max()
    date_range = (df_sorted[date_col] - date_min) / (date_max - date_min + pd.Timedelta(days=1e-9))

    # Compute marker size and color
    sizes = df_sorted[duration_col].fillna(1) * marker_scale + default_marker_size

    # Create scatter trace for cities
    scatter = go.Scattergeo(
        lat=df_sorted[lat_col],
        lon=df_sorted[lon_col],
        text=[f"{row[city_col]}<br>{row[date_col].strftime('%d.%m.%Y')} - {(row[date_col]+timedelta(days=row[duration_col])).strftime('%d.%m.%Y')}:  {row[duration_col]} days" for _, row in df_sorted.iterrows()],
        mode="markers",
        marker=dict(
            size=sizes,
            color=date_range,
            colorscale=color_scale,
            colorbar=dict(title="Date"),
            opacity=0.8,
            line=dict(width=0.5, color="black")
        ),
        name="Cities"
    )

    # Create line segments between cities
    lines = []
    # Loop through cities and create colored lines
    for i in range(len(df_sorted) - 1):
        lat0, lon0 = df_sorted.loc[i, [lat_col, lon_col]]
        lat1, lon1 = df_sorted.loc[i + 1, [lat_col, lon_col]]

        # Normalized color values
        norm0 = date_range.iloc[i]
        norm1 = date_range.iloc[i + 1]

        # Sample colors from scale
        rgb0 = sample_colorscale(color_scale, norm0, colortype='rgb')[0]
        rgb1 = sample_colorscale(color_scale, norm1, colortype='rgb')[0]

        # Convert to RGB tuple and compute midpoint
        def rgb_str_to_tuple(rgb_str):
            return tuple(int(v.strip()) for v in rgb_str.strip("rgb() ").split(","))

        rgb0_tuple = rgb_str_to_tuple(rgb0)
        rgb1_tuple = rgb_str_to_tuple(rgb1)

        mid_rgb = tuple((a + b) // 2 for a, b in zip(rgb0_tuple, rgb1_tuple))
        rgba_color = f"rgba({mid_rgb[0]}, {mid_rgb[1]}, {mid_rgb[2]}, {line_opacity})"

        lines.append(go.Scattergeo(
            lat=[lat0, lat1],
            lon=[lon0, lon1],
            mode="lines",
            line=dict(width=2, color=rgba_color),
            opacity=line_opacity,
            showlegend=False
        ))

    # Calculate bounds if auto_center is enabled


    """
    if auto_center:
        lat_min, lat_max = df_sorted[lat_col].min(), df_sorted[lat_col].max()
        lon_min, lon_max = df_sorted[lon_col].min(), df_sorted[lon_col].max()
        lat_center = (lat_min + lat_max) / 2
        lon_center = (lon_min + lon_max) / 2
        lat_range = lat_max - lat_min
        lon_range = lon_max - lon_min

        geo_scope.update(
            center=dict(lat=lat_center, lon=lon_center),
            #lataxis=dict(range=[lat_min - lat_range * 0.2, lat_max + lat_range * 0.2]),
            #lonaxis=dict(range=[lon_min - lon_range * 0.2, lon_max + lon_range * 0.2])
        )
    """
    fig = go.Figure([scatter] + lines, **plot_kwargs)

    geo_scope = dict(
        projection_type="natural earth",
        resolution=50,
        showland=True,
        landcolor="rgb(243, 243, 243)",
        showcountries=True,
        countrycolor="rgb(204, 204, 204)",
        fitbounds = "locations"
    )
    
    fig.update_layout(
        geo=geo_scope,
        title="Travel Map",
        margin=dict(l=0, r=0, t=40, b=0)
    )

    return fig


In [42]:
extract_headings_only(r"C:\Users\benja\Documents\Reise\Reisebericht.md","reise.md")

df = parse_markdown("reise.md", skip_first_heading=True)
#df = filter_visits(df,country="Australien",start_date="20.06.2025", end_date="12.07.2025")
df

Headings written to: reise.md


,Country,City,Start,End,Duration
0,Neuseeland,Queenstown,2024-02-18,2024-02-18,1
1,24.02-10.03; 14 Tage,01.04-14.04; 14 Tage,2024-04-15,2024-04-15,1
2,24.02-10.03; 14 Tage,21.04-05.05; 14 Tage,2024-05-21,2024-05-21,1
3,Singapur,Singapur,2024-08-08,2024-08-13,6
4,Indonesien,Malang,2024-08-14,2024-08-16,3
...,...,...,...,...,...
81,Australien,West MacDonnell Ranges,2025-07-06,2025-07-07,2
82,Australien,Devils Marbles,2025-07-08,2025-07-08,1
83,Australien,Daly Waters,2025-07-09,2025-07-09,1
84,Australien,Mataranka,2025-07-10,2025-07-10,1


In [33]:
df_summary=summarize_travel(df)
df_summary

,Country,Total_Days,Cities_Visited,Entry_Date,Exit_Date
0,Australien,23,14,2025-06-20,2025-07-12
1,TOTAL,23,14,2025-06-20,2025-07-12


In [34]:
df_geo=map_cities_to_coords(df, city_col="City", country_col="Country", api_wait_time=0.1)
df_geo

,Country,City,Start,End,Duration,Latitude,Longitude
72,Australien,Adelaide,2025-06-20,2025-06-21,2,-34.928181,138.599931
73,Australien,Port Augusta,2025-06-22,2025-06-23,2,-32.490882,137.763987
74,Australien,Marree,2025-06-24,2025-06-24,1,-29.647510,138.063420
75,Australien,William Creek,2025-06-25,2025-06-25,1,-28.907794,136.341327
76,Australien,Oodnadatta,2025-06-26,2025-06-26,1,-27.548276,135.448427
77,Australien,Coober Pedy,2025-06-27,2025-06-29,3,-29.013368,134.753616
78,Australien,Uluru/Ayers Rock,2025-06-30,2025-07-02,3,-25.345554,131.036961
79,Australien,Kings Canyon,2025-07-03,2025-07-03,1,-24.250309,131.577950
80,Australien,Alice Springs,2025-07-04,2025-07-05,2,-23.698388,133.881289
81,Australien,West MacDonnell Ranges,2025-07-06,2025-07-07,2,NaN,NaN


In [35]:
unmapped=get_unmapped_locations(df_geo)
unmapped

,Country,City,Start,End,Duration,Latitude,Longitude
81,Australien,West MacDonnell Ranges,2025-07-06,2025-07-07,2,NaN,NaN


In [36]:
updates=[
    #[13,34.816667, 134.683333],
     #    [75,-29.011111,134.755556],
         [81, -23.750082, 132.97867]]
df_geo=set_manual_coordinates_by_index(df_geo,updates)

In [37]:
df_geo

,Country,City,Start,End,Duration,Latitude,Longitude
72,Australien,Adelaide,2025-06-20,2025-06-21,2,-34.928181,138.599931
73,Australien,Port Augusta,2025-06-22,2025-06-23,2,-32.490882,137.763987
74,Australien,Marree,2025-06-24,2025-06-24,1,-29.647510,138.063420
75,Australien,William Creek,2025-06-25,2025-06-25,1,-28.907794,136.341327
76,Australien,Oodnadatta,2025-06-26,2025-06-26,1,-27.548276,135.448427
77,Australien,Coober Pedy,2025-06-27,2025-06-29,3,-29.013368,134.753616
78,Australien,Uluru/Ayers Rock,2025-06-30,2025-07-02,3,-25.345554,131.036961
79,Australien,Kings Canyon,2025-07-03,2025-07-03,1,-24.250309,131.577950
80,Australien,Alice Springs,2025-07-04,2025-07-05,2,-23.698388,133.881289
81,Australien,West MacDonnell Ranges,2025-07-06,2025-07-07,2,-23.750082,132.978670


In [16]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from typing import Optional, Dict

def plot_travel_map(
    df: pd.DataFrame,
    lat_col: str = "Latitude",
    lon_col: str = "Longitude",
    city_col: str = "City",
    duration_col: str = "Duration",
    date_col: str = "Start",
    default_marker_size: int = 5,
    marker_scale: float = 2.0,
    color_scale: str = "Viridis",
    line_opacity: float = 0.6,
    plot_kwargs: Optional[Dict] = None
) -> go.Figure:
    """
    Plots a world map of visited cities with size based on duration and color based on entry date.

    Parameters:
        df (pd.DataFrame): DataFrame with geolocation and visit data.
        lat_col (str): Name of the latitude column.
        lon_col (str): Name of the longitude column.
        city_col (str): Name of the city column.
        duration_col (str): Name of the duration column.
        date_col (str): Name of the date column (should be datetime).
        default_marker_size (int): Minimum size for markers.
        marker_scale (float): Scaling factor for marker size.
        color_scale (str): Color scale name for Plotly.
        line_opacity (float): Opacity of the lines connecting cities.
        plot_kwargs (dict): Additional keyword arguments to pass to `go.Figure`.

    Returns:
        go.Figure: The resulting Plotly figure.
    """
    if plot_kwargs is None:
        plot_kwargs = {}

    df_sorted = df.sort_values(by=date_col).reset_index(drop=True)

    # Normalize date to float for coloring
    date_min = df_sorted[date_col].min()
    date_max = df_sorted[date_col].max()
    date_range = (df_sorted[date_col] - date_min) / (date_max - date_min + pd.Timedelta(days=1e-9))

    # Compute marker size and color
    sizes = df_sorted[duration_col].fillna(1) * marker_scale + default_marker_size
    colors = date_range

    # Create scatter trace for cities
    scatter = go.Scattergeo(
        lat=df_sorted[lat_col],
        lon=df_sorted[lon_col],
        text=[f"{row[city_col]}<br>{row[date_col].strftime('%d.%m.%Y')} - {(row[date_col]+timedelta(days=row[duration_col])).strftime('%d.%m.%Y')}:  {row[duration_col]} days" for _, row in df_sorted.iterrows()],
        mode="markers",
        marker=dict(
            size=sizes,
            color=colors,
            colorscale=color_scale,
            colorbar=dict(title="Date"),
            opacity=0.8,
            line=dict(width=0.5, color="black")
        ),
        name="Cities"
    )



import plotly.graph_objects as go
from plotly.colors import sample_colorscale
import pandas as pd
import numpy as np
from typing import Optional, Dict

import plotly.colors as pc

def get_rgb_from_scale(value: float, colorscale: str) -> tuple:
    """
    Convert a normalized value (0-1) into an RGB tuple using the given Plotly colorscale.

    Parameters:
        value (float): Normalized value between 0 and 1.
        colorscale (str): Name of the Plotly color scale.

    Returns:
        tuple: (r, g, b)
    """
    # Ensure value is in [0, 1]
    value = max(0, min(1, value))
    rgb_string = pc.find_intermediate_color(
        pc.get_colorscale(colorscale)[0][1],
        pc.get_colorscale(colorscale)[-1][1],
        value,
        colortype="rgb"
    )
    return tuple(int(x) for x in rgb_string.strip("rgb()").split(","))

import plotly.graph_objects as go

def midpoint_rgb(color1: str, color2: str, alpha: float = 1.0) -> str:
    """
    Compute the midpoint RGBA color string between two RGB strings.
    
    Parameters:
        color1 (str): RGB color string, e.g., 'rgb(255, 100, 0)'
        color2 (str): RGB color string
        alpha (float): Alpha (opacity) value for the output RGBA string
    
    Returns:
        str: RGBA color string
    """
    def rgb_str_to_tuple(rgb_str):
        return tuple(int(c.strip()) for c in rgb_str.strip("rgb() ").split(","))

    rgb1 = rgb_str_to_tuple(color1)
    rgb2 = rgb_str_to_tuple(color2)
    mid_rgb = tuple((a + b) // 2 for a, b in zip(rgb1, rgb2))

    return f"rgba({mid_rgb[0]}, {mid_rgb[1]}, {mid_rgb[2]}, {alpha})"

def build_colored_lines_geo(df_sorted, lat_col, lon_col, date_range, color_scale, width=2, line_opacity=0.8, **kwargs):
    """
    Builds a list of Scattergeo line traces colored by the midpoint of a colorscale.

    Parameters:
        df_sorted (pd.DataFrame): Sorted DataFrame with lat/lon columns.
        lat_col (str): Name of the latitude column.
        lon_col (str): Name of the longitude column.
        date_range (pd.Series): Normalized values (0 to 1) for the colorscale.
        color_scale (list): Plotly-compatible colorscale.
        line_opacity (float): Line opacity for RGBA colors.
        width (int): width of the plotted lines
        **kwargs: Additional arguments to pass to go.Scattergeo (e.g. name, hoverinfo).

    Returns:
        list: List of plotly.graph_objects.Scattergeo traces.
    """
    lines = []

    for i in range(len(df_sorted) - 1):
        lat0, lon0 = df_sorted.loc[i, [lat_col, lon_col]]
        lat1, lon1 = df_sorted.loc[i + 1, [lat_col, lon_col]]

        norm0 = date_range.iloc[i]
        norm1 = date_range.iloc[i + 1]

        rgb0 = sample_colorscale(color_scale, norm0, colortype='rgb')[0]
        rgb1 = sample_colorscale(color_scale, norm1, colortype='rgb')[0]

        rgba_color = midpoint_rgb(rgb0, rgb1, alpha=line_opacity)

        lines.append(go.Scattergeo(
            lat=[lat0, lat1],
            lon=[lon0, lon1],
            mode="lines",
            line=dict(width=width, color=rgba_color),
            opacity=line_opacity,
            **kwargs
        ))

    return lines



def plot_travel_map_hist(
    df: pd.DataFrame,
    lat_col: str = "Latitude",
    lon_col: str = "Longitude",
    city_col: str = "City",
    duration_col: str = "Duration",
    date_col: str = "Start",
    default_marker_size: int = 5,
    marker_scale: float = 2.0,
    color_scale: str = "Viridis",
    line_opacity: float = 0.6,
    auto_center: bool = False,
    plot_kwargs: Optional[Dict] = None
) -> go.Figure:
    """
    Plots a world map of visited cities with size based on duration and color based on entry date.

    Parameters:
        df (pd.DataFrame): DataFrame with geolocation and visit data.
        lat_col (str): Name of the latitude column.
        lon_col (str): Name of the longitude column.
        city_col (str): Name of the city column.
        duration_col (str): Name of the duration column.
        date_col (str): Name of the entry date column (should be datetime).
        default_marker_size (int): Minimum size for markers.
        marker_scale (float): Scaling factor for marker size.
        color_scale (str): Color scale name for Plotly.
        line_opacity (float): Opacity of the lines connecting cities.
        auto_center (bool): If True, zoom to the bounding box of the cities.
        plot_kwargs (dict): Additional keyword arguments to pass to `go.Figure`.

    Returns:
        go.Figure: The resulting Plotly figure.
    """
    if plot_kwargs is None:
        plot_kwargs = {}

    df_sorted = df.sort_values(by=date_col).reset_index(drop=True)

    # Normalize date to float for coloring
    date_min = df_sorted[date_col].min()
    date_max = df_sorted[date_col].max()
    date_range = (df_sorted[date_col] - date_min) / (date_max - date_min + pd.Timedelta(days=1e-9))

    # Compute marker size and color
    sizes = df_sorted[duration_col].fillna(1) * marker_scale + default_marker_size

    # Create scatter trace for cities
    scatter = go.Scattergeo(
        lat=df_sorted[lat_col],
        lon=df_sorted[lon_col],
        text=[f"{row[city_col]}<br>{row[date_col].strftime('%d.%m.%Y')} - {(row[date_col]+timedelta(days=row[duration_col])).strftime('%d.%m.%Y')}:  {row[duration_col]} days" for _, row in df_sorted.iterrows()],
        mode="markers",
        marker=dict(
            size=sizes,
            color=date_range,
            colorscale=color_scale,
            colorbar=dict(title="Date"),
            opacity=line_opacity,
            line=dict(width=0.5, color="black")
        ),
        name="Cities"
    )

    lines = build_colored_lines_geo(
        df_sorted=df_sorted,
        lat_col="Latitude",
        lon_col="Longitude",
        date_range=date_range,
        color_scale=color_scale,
        line_opacity=line_opacity,
        width=2,
        showlegend=False
    )

    fig = go.Figure([scatter] + lines, **plot_kwargs)

    fig.update_layout(
        geo=dict(
            projection_type="natural earth",
            showland=True,
            landcolor="rgb(243, 243, 243)",
            showcountries=True,
            countrycolor="rgb(204, 204, 204)",
            fitbounds="locations"
        ),
        title="Travel Map",
        margin=dict(l=0, r=0, t=40, b=0)
    )

    return fig


In [39]:
def plot_travel_map(
    df: pd.DataFrame,
    lat_col: str = "Latitude",
    lon_col: str = "Longitude",
    city_col: str = "City",
    duration_col: str = "Duration",
    date_col: str = "Start",
    default_marker_size: int = 5,
    marker_scale: float = 2.0,
    color_scale: str = "Viridis",
    line_opacity: float = 0.6,
    location_name: str = "Cities",
    city_marker_kwargs: Optional[Dict] = None,
    line_kwargs: Optional[Dict] = None,
    plot_kwargs: Optional[Dict] = None
) -> go.Figure:
    """
    Plots a world map of visited cities with size based on duration and color based on entry date.

    Parameters:
        df (pd.DataFrame): DataFrame with geolocation and visit data.
        lat_col (str): Name of the latitude column.
        lon_col (str): Name of the longitude column.
        city_col (str): Name of the city column.
        duration_col (str): Name of the duration column.
        date_col (str): Name of the entry date column (should be datetime).
        default_marker_size (int): Minimum size for markers.
        marker_scale (float): Scaling factor for marker size.
        color_scale (str): Color scale name for Plotly.
        line_opacity (float): Opacity of the lines connecting cities.
        location_name (string): Name for the Location plots
        city_marker_kwargs (dict): Extra kwargs for the Scattergeo markers.
        line_kwargs (dict): Extra kwargs for the lines (passed to build_colored_lines_geo).
        plot_kwargs (dict): Extra kwargs for the go.Figure/layout.

    Returns:
        go.Figure: The resulting Plotly figure.
    """

    # ---------- Default configurations ----------
    default_city_marker_kwargs = dict(
        colorbar=dict(title="Date"),
        line=dict(width=0.5, color="black")
    )

    default_line_kwargs = dict(
        showlegend=False,
        width=2,
        lat_col="Latitude",
        lon_col="Longitude",
    )

    default_plot_kwargs = dict(
        geo=dict(
            projection_type="natural earth",
            showland=True,
            landcolor="rgb(243, 243, 243)",
            showcountries=True,
            countrycolor="rgb(204, 204, 204)",
            fitbounds="locations"
        ),
        title="Travel Map",
        margin=dict(l=0, r=0, t=40, b=0)
    )

    # ---------- Apply user overrides ----------
    city_kwargs = {**default_city_marker_kwargs, **(city_marker_kwargs or {})}
    line_kwargs = {**default_line_kwargs, **(line_kwargs or {})}
    plot_kwargs = {**default_plot_kwargs, **(plot_kwargs or {})}

    df_sorted = df.sort_values(by=date_col).reset_index(drop=True)

    # Normalize date to float for coloring
    date_min = df_sorted[date_col].min()
    date_max = df_sorted[date_col].max()
    date_range = (df_sorted[date_col] - date_min) / (date_max - date_min + pd.Timedelta(days=1e-9))

    # Compute marker size and color
    sizes = df_sorted[duration_col].fillna(1) * marker_scale + default_marker_size

    # Create scatter trace for cities
    scatter = go.Scattergeo(
        lat=df_sorted[lat_col],
        lon=df_sorted[lon_col],
        text=[f"{row[city_col]}<br>{row[date_col].strftime('%d.%m.%Y')} - {(row[date_col]+timedelta(days=row[duration_col])).strftime('%d.%m.%Y')}:  {row[duration_col]} days" for _, row in df_sorted.iterrows()],
        mode="markers",
        marker=dict(
            size=sizes,
            color=date_range,
            colorscale=color_scale,
            opacity=line_opacity,
            **city_kwargs
        ),
        name=location_name
    )

    lines = build_colored_lines_geo(
        df_sorted=df_sorted,
        date_range=date_range,
        color_scale=color_scale,
        line_opacity=line_opacity,
        **line_kwargs
    )

    fig = go.Figure([scatter] + lines)

    fig.update_layout(
        **plot_kwargs
    )

    return fig

In [40]:
mad_max_colors = [
    "#726250",  # dusty grey-brown (neutral midtone)
    "#9c1c13",  # blood red (deep vivid accent)
    "#c85d17",  # burnt copper (warm midtone)
    "#d94f04",  # rusty orange (bright highlight)
    "#e7a31d",  # sand yellow (lightest tone)
]


def build_continuous_scale(hex_colors):
    """Convert hex color list to Plotly continuous color scale format."""
    n = len(hex_colors)
    return [[i / (n - 1), color] for i, color in enumerate(hex_colors)]

mad_max_scale = build_continuous_scale(mad_max_colors)

fig = plot_travel_map(df_geo, line_opacity=0.8, color_scale=mad_max_scale)
fig.show()

In [19]:
fig.update_geos(
    projection_type="natural earth",  # gives a flat, map-colored globe
    showland=True,
    landcolor="rgb(243, 231, 207)",
    showocean=True,
    oceancolor="lightblue",
    showcountries=True,
)
fig.show()

In [41]:
fig = plot_travel_map(df_geo, line_opacity=0.8)
fig.show()


In [ ]:
mad_max_colors = [
    "#2c1d0b",  # dark burnt brown
    "#d94f04",  # rusty orange
    "#e7a31d",  # sand yellow
    "#726250",  # dusty grey-brown
    "#3b3b3b",  # dark steel grey
    "#9c1c13",  # blood red
    "#c85d17",  # burnt copper
]


def build_continuous_scale(hex_colors):
    """Convert hex color list to Plotly continuous color scale format."""
    n = len(hex_colors)
    return [[i / (n - 1), color] for i, color in enumerate(hex_colors)]

mad_max_scale = build_continuous_scale(mad_max_colors)

import plotly.graph_objects as go
import numpy as np

z = np.random.rand(10, 10)

fig = go.Figure(data=go.Heatmap(
    z=z,
    colorscale=mad_max_scale,
    colorbar=dict(title="Intensity")
))
fig.update_layout(title="Mad Max Style Heatmap")
fig.show(renderer="iframe_connected")